# FINAL MODEL 1
#### Name: Manuel Alejandro Delgadillo Ferrel
#### Student number: S4400151

In [1]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from model import Model
from dmchunk import Chunk
import math
import random

In [2]:
# Time conversion functions
def noise(s):
    rand = random.uniform(0.001,0.999)
    return s * math.log((1 - rand)/rand)

def time_to_pulses(time, t_0 = 0.011, a = 1.1, b = 0.015):
    pulses = 0
    pulse_duration = t_0
    while time >= pulse_duration:
        time = time - pulse_duration
        pulses += 1
        pulse_duration = a * pulse_duration + noise(b * a * pulse_duration)
    return pulses

def pulses_to_time(pulses, t_0 = 0.011, a = 1.1, b = 0.015):
    time = 0
    pulse_duration = t_0
    while pulses > 0:
        time = time + pulse_duration
        pulses = pulses - 1
        pulse_duration = a * pulse_duration + noise(b * a * pulse_duration)
    return time

#Functions for standard deviation
def variance(data, ddof=0):
    n = len(data)
    mean = sum(data)/n
    return sum((x-mean)**2 for x in data) / (n-ddof)
def stdev(data):
    var = variance(data)
    std_dev = math.sqrt(var)
    return std_dev

In [3]:
#Setting up initial values
participants = 10
conditions = 2
TrainingTrials = 500
MaxTrials = 1500

short_inter = np.linspace(450, 825, num=6)
long_inter = np.linspace(750, 1125, num=6)
medium_inter = np.linspace(600, 975, num=6)

#Only for probability values
i = 1/6
chance = [i for j in range(6)]

#The 675 ms interval occurred with more frequency
probability = {"Short Uniform": chance,
               "Long Uniform": chance,
               "Uniform": chance,
               "Peaked": [1/12, 7/12, 1/12, 1/12, 1/12, 1/12]}



In [4]:
#Only for observing purposes
short_inter

array([450., 525., 600., 675., 750., 825.])

In [5]:
long_inter

array([ 750.,  825.,  900.,  975., 1050., 1125.])

In [6]:
medium_inter

array([600., 675., 750., 825., 900., 975.])

In [7]:
#Heading of the dataframe
count = 0
df = pd.DataFrame(columns = ["int", "resp","Subject", "Block", "Run", "Task", "Main"])

for Subject in range(1, participants+1):
    if Subject <= 4:
        conditions = {"Short Uniform": short_inter,
                      "Long Uniform": long_inter}
        Task=1
    else:
        conditions = {"Uniform": medium_inter,
                      "Peaked": medium_inter}
        Task=2
    for cond in conditions:
        m = Model()
        main = False
                
        #trials ranging from 500 to 1500
        train_trials = np.random.randint(TrainingTrials, MaxTrials)
            
        #sessions were divided into 2 trials
        TestTrials = train_trials + MaxTrials - TrainingTrials
            
        for trial in range(TestTrials):
            #initial 1s
            m.time += 1
               
            #interval for each condition (short, long, uniform and peaked)
            interval = np.random.choice(conditions[cond], p = probability[cond])
            m.time += interval /1000
            pulses = time_to_pulses(interval / 1000)
                
            #Yellow dot appears on the screen for at least 185 ms which varies from trial to trial
            m.time += np.random.uniform(0.0185, 0.02)
            
            #Input Chunk
            fact = Chunk(name = "tf" + str(pulses), slots ={"isa":"time-fact", "pulses" : pulses})
            m.add_encounter(fact)
                
            #wait for at least 250ms after the flash
            m.time += np.random.uniform(0.25, 1)
                
            #time interval
            pattern = Chunk(name = "retreiving-goal", slots = {"isa":"time-fact"})
            t,latency = m.retrieve_blended_trace(pattern, "pulses")
            r_prob = m.get_retrieval_probability(fact, pattern)
            print(r_prob)

            time = pulses_to_time(t)
            m.time += pulses_to_time(t)
            time = (pulses_to_time(t))*1000
                
            #delay of 450ms-850ms before feedback
            m.time += np.random.uniform(0.45, 0.85)
                
            #displaying of the performance for 62ms
            m.time += 0.062
                
            #Recording of the training was necessary for observing purposes
            if trial > train_trials:
                main = True
                df.loc[count] = [interval,time,Subject,cond,trial + 1,Task,main]
                count += 1
                

{<dmchunk.Chunk object at 0x0000025D3722B4F0>: 0.4518594442752588}
[<dmchunk.Chunk object at 0x0000025D3722B4F0>, <dmchunk.Chunk object at 0x0000025D3722B700>]
1.0
{<dmchunk.Chunk object at 0x0000025D3722B4F0>: -0.682887008275895, <dmchunk.Chunk object at 0x0000025D3722B970>: 0.16474313138843336}
[<dmchunk.Chunk object at 0x0000025D3722B4F0>, <dmchunk.Chunk object at 0x0000025D3722B700>, <dmchunk.Chunk object at 0x0000025D3722B970>]
0.9857711228407678
{<dmchunk.Chunk object at 0x0000025D3722B4F0>: -1.0299923995536162, <dmchunk.Chunk object at 0x0000025D3722B970>: -0.7680451419393013, <dmchunk.Chunk object at 0x0000025D3722BBE0>: 0.025208071191938557}
[<dmchunk.Chunk object at 0x0000025D3722B4F0>, <dmchunk.Chunk object at 0x0000025D3722B700>, <dmchunk.Chunk object at 0x0000025D3722B970>, <dmchunk.Chunk object at 0x0000025D3722BBE0>]
0.9765082710780166
{<dmchunk.Chunk object at 0x0000025D3722B4F0>: -1.2389263071218553, <dmchunk.Chunk object at 0x0000025D3722B970>: -1.0825909665538465, <d

KeyError: <dmchunk.Chunk object at 0x0000025D34521040>

In [ ]:
df.to_csv("model.csv", index = False)

In [ ]:
df.head()

In [ ]:
#Plotting of the data found during the experiments for response bias values
def plot_exp1(df, interval_1, interval_2):
    df = df[df["Main"] == True]
    intervals = df[df["Block"].isin([interval_1, interval_2])]

    fig = plt.figure(figsize = (20, 7))
    ax1 = fig.add_subplot(131)
    ax2 = fig.add_subplot(132)

    #Experiment for single subjects
    first_subject = intervals["Subject"].unique()[0]
    X = intervals[intervals["Subject"] == first_subject]

    a1 = X[X["Block"] == interval_1]    
    a1 = a1.groupby(["int"])["resp"].mean().reset_index()
    a1["bias"] = a1["resp"] - a1["int"]

    a2 = X[X["Block"] == interval_2]    
    a2 = a2.groupby(["int"])["resp"].mean().reset_index()
    a2["bias"] = a2["resp"] - a2["int"]

    ax1.scatter(a1["int"], a1["bias"], s=50, c='firebrick', marker=".", label=interval_1)
    ax1.scatter(a2["int"], a2["bias"], s=50, c='lawngreen', marker=".", label=interval_2)
    ax1.axhline(y=0, color='gray', linestyle='-')
    ax1.set(xlabel = "Physical time interval (ms)", ylabel = "Response Bias (ms)")
    ax1.set_title("Single Subject")

    #Experiment for a group of subjects
    a1 = intervals[intervals["Block"] == interval_1]    
    a1 = a1.groupby(["int"])["resp"].mean().reset_index()
    a1["bias"] = a1["resp"] - a1["int"]

    a2 = intervals[intervals["Block"] == interval_2]    
    a2 = a2.groupby(["int"])["resp"].mean().reset_index()
    a2["bias"] = a2["resp"] - a2["int"]

    ax2.scatter(a1["int"], a1["bias"], s=50, c='firebrick', marker=".", label=interval_1)
    ax2.scatter(a2["int"], a2["bias"], s=50, c='lawngreen', marker=".", label=interval_2)
    ax2.axhline(y=0, color='gray', linestyle='-')
    ax2.set(xlabel = "Physical time interval (ms)", ylabel = "Response Bias (ms)")
    ax2.set_title("Group Mean")
    
    ax1.legend(loc='upper right')
    ax2.legend(loc = 'upper right')
    plt.show()

In [ ]:
#Plotting of the data found during the experiments for response standard deviation values
def plot_exp2(df, interval_1, interval_2):
    df = df[df["Main"] == True]
    intervals = df[df["Block"].isin([interval_1, interval_2])]

    fig = plt.figure(figsize = (20, 7))
    ax1 = fig.add_subplot(131)
    ax2 = fig.add_subplot(132)

    #Experiment for single subjects
    first_subject = intervals["Subject"].unique()[0]
    X = intervals[intervals["Subject"] == first_subject]

    a1 = X[X["Block"] == interval_1]    
    a1 = a1.groupby(["int"])["resp"].std().reset_index()
    a1["bias"] =stdev(a1["resp"] - a1["int"])
    
    a2 = X[X["Block"] == interval_2]    
    a2 = a2.groupby(["int"])["resp"].std().reset_index()
    a2["bias"] =stdev(a1["resp"] - a1["int"])
    
    ax1.scatter(a1["int"], a1["bias"], s=50, c='firebrick', marker=".", label=interval_1)
    ax1.scatter(a2["int"], a2["bias"], s=50, c='lawngreen', marker=".", label=interval_2)
    ax1.axhline(y=0, color='gray', linestyle='-')
    ax1.set(xlabel = "Physical time interval (ms)", ylabel = "Response sd (ms)")
    ax1.set_title("Single Subject")

    #Experiment for a group of subjects
    a1 = intervals[intervals["Block"] == interval_1]    
    a1 = a1.groupby(["int"])["resp"].std().reset_index()
    a1["bias"] =stdev(a1["resp"] - a1["int"])
    
    a2 = intervals[intervals["Block"] == interval_2]    
    a2 = a2.groupby(["int"])["resp"].std().reset_index()
    a2["bias"] =stdev(a1["resp"] - a1["int"])
    
    ax2.scatter(a1["int"], a1["bias"], s=50, c='firebrick', marker=".", label=interval_1)
    ax2.scatter(a2["int"], a2["bias"], s=50, c='lawngreen', marker=".", label=interval_2)
    ax2.axhline(y=0, color='gray', linestyle='-')
    ax2.set(xlabel = "Physical time interval (ms)", ylabel = "Response sd (ms)")
    ax2.set_title("Group Mean")
    
    ax1.legend(loc='center right')
    ax2.legend(loc = 'center right')
    plt.show()

In [ ]:
plot_exp1(df, "Short Uniform", "Long Uniform")
plot_exp2(df, "Short Uniform", "Long Uniform")

In [ ]:
plot_exp1(df, "Uniform", "Peaked")
plot_exp2(df, "Uniform", "Peaked")

In [ ]:
#I tried using the standard deviation mean but it vary from -80 to 100 so it didnt make much sense compared to the Acerbi
#experiment, so I left it as the mean standard deviation 